<a href="https://colab.research.google.com/github/Scyzentraz/modulajar/blob/main/Nonstable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#Langkah 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Langkah2. Install Model dan Dependensi yang dibutuhkan

#Install Real esrgan
import os

if not os.path.exists("Real-ESRGAN"):
    !git clone https://github.com/xinntao/Real-ESRGAN.git

#Install dependensi
%cd Real-ESRGAN
!pip install basicsr facexlib gfpgan ffmpeg-python
!pip install -r requirements.txt
!python setup.py develop
!pip install tqdm
!pip install pillow

# Unduh waifu2x-ncnn-vulkan
!wget https://github.com/nihui/waifu2x-ncnn-vulkan/releases/download/20220728/waifu2x-ncnn-vulkan-20220728-ubuntu.zip

# Ekstrak ke folder waifu2x
!unzip waifu2x-ncnn-vulkan-20220728-ubuntu.zip -d waifu2x

# Ubah permission ke binary yang benar
!chmod +x waifu2x/waifu2x-ncnn-vulkan-20220728-ubuntu/waifu2x-ncnn-vulkan

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 759 (delta 106), reused 99 (delta 99), pack-reused 638 (from 1)
Receiving objects: 100% (759/759), 5.38 MiB | 42.69 MiB/s, done.
Resolving deltas: 100% (415/415), done.
/content/Real-ESRGAN
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Usin

In [5]:
#Langkah 3. Patch bug rgb to grayscale (opsional)
file_path = "/usr/local/lib/python3.11/dist-packages/basicsr/data/degradations.py"

with open(file_path, 'r') as file:
    file_data = file.readlines()

# Hanya patch jika belum dipatch
patched = False
for i, line in enumerate(file_data):
    if 'from torchvision.transforms.functional_tensor import rgb_to_grayscale' in line:
        file_data[i] = 'from torchvision.transforms.functional import rgb_to_grayscale\n'
        patched = True
        break

if patched:
    with open(file_path, 'w') as file:
        file.writelines(file_data)
    print("✅ Bug torchvision berhasil dipatch.")
else:
    print("✅ Tidak perlu patch, sudah sesuai.")

✅ Bug torchvision berhasil dipatch.


In [4]:
#Langkah 4.1 siapkan file input (manual di google drive, masukan gambar awal ke folder /upscale/input)
#Langkah 4.2 konfigurasi batch
import os
from glob import glob




# Konfigurasi
source = "image"
input_dir = "/content/drive/MyDrive/upscale/input"
output_dir_waifu2x = "/content/drive/MyDrive/upscale/waifu2x_denoise_alt"
output_dir_1 = "/content/drive/MyDrive/upscale/output1"
output_dir_2 = "/content/drive/MyDrive/upscale/output2"
output_dir_3 = "/content/drive/MyDrive/upscale/output3"
final_output_dir = "/content/drive/MyDrive/upscale/final_jpeg"
model = "RealESRGAN_x2plus"
scale = 2
face_enhance = False


# Buat folder output jika belum ada
os.makedirs(output_dir_waifu2x, exist_ok=True)
os.makedirs(output_dir_1, exist_ok=True)
os.makedirs(output_dir_2, exist_ok=True)
os.makedirs(output_dir_3, exist_ok=True)
os.makedirs(final_output_dir, exist_ok=True)

# Ambil semua file jpg/jpeg
image_paths = sorted(
    glob(os.path.join(input_dir, "*.jpg")) +
    glob(os.path.join(input_dir, "*.jpeg")) +
    glob(os.path.join(input_dir, "*.png"))
)
print(f"Jumlah gambar ditemukan: {len(image_paths)}")

Jumlah gambar ditemukan: 1


In [3]:
import os
import cv2
from glob import glob
from tqdm import tqdm

input_dir = "/content/drive/MyDrive/upscale/input"
output_dir = "/content/drive/MyDrive/upscale/waifu2x_denoise_alt"
os.makedirs(output_dir, exist_ok=True)
image_paths = sorted(glob(os.path.join(input_dir, "*.jpg")) + glob(os.path.join(input_dir, "*.png")))

for img_path in tqdm(image_paths, desc="🔇 Denoising OpenCV"):
    filename = os.path.basename(img_path)
    img = cv2.imread(img_path)
    denoised = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)
    out_path = os.path.join(output_dir, filename)
    cv2.imwrite(out_path, denoised)

🔇 Denoising OpenCV: 100%|██████████| 1/1 [00:06<00:00,  6.93s/it]


In [11]:

#Langkah 5.1 Ambil semua file jpg/jpeg/png, gunakan output dari waifu2x sebagai input ke tahap upscale pertama
image_paths = sorted(
    glob(os.path.join(output_dir_waifu2x, "*.jpg")) +
    glob(os.path.join(output_dir_waifu2x, "*.jpeg")) +
    glob(os.path.join(output_dir_waifu2x, "*.png"))
)
print(f"Jumlah gambar ditemukan: {len(image_paths)}")
if not os.listdir(output_dir_waifu2x):
    print("❌ Tidak ada output dari tahap preprocess!")

Jumlah gambar ditemukan: 1


In [15]:

from tqdm import tqdm
import os

code = "inference_realesrgan.py"
tile = 0  # Sesuaikan jika kehabisan memori

for input_path in tqdm(image_paths, desc="🚀 Memproses gambar"):
    command = f'python {code} -i "{input_path}" -n {model} -o "{output_dir_1}" -s {scale} --tile {tile}'

    if face_enhance:
        command += " --face_enhance"

    os.system(command)

🚀 Memproses gambar: 100%|██████████| 1/1 [00:00<00:00, 19.58it/s]


In [22]:
!python inference_realesrgan.py -i "/content/drive/MyDrive/upscale/waifu2x_denoise_alt/20250630_095343.jpg" -n RealESRGAN_x4plus -o "/content/drive/MyDrive/upscale/output1" -s 2 --tile 0

Downloading: "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth" to /content/Real-ESRGAN/weights/RealESRGAN_x4plus.pth

100% 63.9M/63.9M [00:00<00:00, 336MB/s]
Testing 0 20250630_095343


In [23]:

#Langkah 5.1.1 Upscale tahap pertama
from tqdm import tqdm

code = "inference_realesrgan.py"
tile = 0  # ukuran tile, bisa disesuaikan jika masih kehabisan memori

# Jalankan batch dengan progress bar

for input_path in tqdm(image_paths, desc="🚀 Memproses gambar"):
    filename = os.path.splitext(os.path.basename(input_path))[0] + ".png"
    output_path = os.path.join(output_dir_1, filename)
    command = f'python {code} -i "{input_path}" -n {model} -o "{output_path}" -s {scale} --tile {tile}'

    if face_enhance:
        command += " --face_enhance"

    os.system(command)

🚀 Memproses gambar: 100%|██████████| 1/1 [00:09<00:00,  9.92s/it]


In [21]:
%cd /content/Real-ESRGAN

/content/Real-ESRGAN


In [24]:
# Langkah 5.2 Ambil semua file jpg/jpeg/png, gunakan output dari tahap pertama sebagai input ke tahap upscale kedua

image_paths = sorted(
    glob(os.path.join(output_dir_1, "*.jpg")) +
    glob(os.path.join(output_dir_1, "*.jpeg")) +
    glob(os.path.join(output_dir_1, "*.png"))
)
print(f"Jumlah gambar ditemukan: {len(image_paths)}")
if not os.listdir(output_dir_1):
    print("❌ Tidak ada output dari tahap pertama!")

Jumlah gambar ditemukan: 1


In [25]:
#Langkah 5.2.1 upscale tahap 2
from tqdm import tqdm
import os

code = "inference_realesrgan.py"
tile = 400  # ukuran tile, bisa disesuaikan kalau masih kehabisan memori

# Jalankan batch dengan progress bar

for input_path in tqdm(image_paths, desc="🚀 Memproses gambar"):
    filename = os.path.splitext(os.path.basename(input_path))[0] + ".png"
    output_path = os.path.join(output_dir_2, filename)
    command = f'python {code} -i "{input_path}" -n {model} -o "{output_path}" -s {scale} --tile {tile}'


    if face_enhance:
        command += " --face_enhance"

    print("\n[📤 Menjalankan command:]")
    print(command)  # log command yang dijalankan

    exit_code = os.system(command)

    if exit_code != 0:
        print(f"[❌ Gagal memproses {input_path}] (exit code: {exit_code})")
    else:
        print(f"[✅ Berhasil memproses {input_path}]")

🚀 Memproses gambar:   0%|          | 0/1 [00:00<?, ?it/s]


[📤 Menjalankan command:]
python inference_realesrgan.py -i "/content/drive/MyDrive/upscale/output1/20250630_095343.png" -n RealESRGAN_x2plus -o "/content/drive/MyDrive/upscale/output2/20250630_095343.png" -s 2 --tile 400


🚀 Memproses gambar: 100%|██████████| 1/1 [00:20<00:00, 20.02s/it]

[✅ Berhasil memproses /content/drive/MyDrive/upscale/output1/20250630_095343.png]


In [26]:
#Langkah 5.3 Ambil semua file jpg/jpeg/png, gunakan output dari tahap kedua sebagai input ke tahap upscale ketiga

image_paths = sorted(
    glob(os.path.join(output_dir_2, "*.jpg")) +
    glob(os.path.join(output_dir_2, "*.jpeg")) +
    glob(os.path.join(output_dir_2, "*.png"))
)
print(f"Jumlah gambar ditemukan: {len(image_paths)}")
if not os.listdir(output_dir_2):
    print("❌ Tidak ada output dari tahap kedua!")

Jumlah gambar ditemukan: 1


In [37]:
#Langkah 5.3.1 upscale tahap 3
from tqdm import tqdm
import subprocess
import os

tile = 200  #
tile_pad = 10
code = "inference_realesrgan.py"

for input_path in tqdm(image_paths, desc="🚀 Memproses gambar"):
    filename = os.path.splitext(os.path.basename(input_path))[0] + ".png"
    command = f'python {code} -i "{input_path}" -n {model} -o "{output_dir_3}" -s 2 --tile {tile} --tile_pad {tile_pad}'

    if face_enhance:
        command += " --face_enhance"

    print("[📤 Menjalankan command:]")
    print(command)

    result = subprocess.run(command, shell=True, capture_output=True, text=True)

    if result.returncode != 0:
        print(f"[❌ Gagal memproses {input_path}]")
        print("STDERR:", result.stderr)
    else:
        print(f"[✅ Berhasil memproses {input_path}]")

🚀 Memproses gambar:   0%|          | 0/1 [00:00<?, ?it/s]

[📤 Menjalankan command:]
python inference_realesrgan.py -i "/content/drive/MyDrive/upscale/output3/20250630_095343.png" -n RealESRGAN_x2plus -o "/content/drive/MyDrive/upscale/output3" -s 2 --tile 200 --tile_pad 10


🚀 Memproses gambar: 100%|██████████| 1/1 [00:05<00:00,  5.99s/it]

[✅ Berhasil memproses /content/drive/MyDrive/upscale/output3/20250630_095343.png]


In [34]:
#Langkah 6. Ambil semua file jpg/jpeg/png, gunakan output dari tahap ketiga sebagai input ke tahap jpeg 100%

image_paths = sorted(
    glob(os.path.join(output_dir_3, "*.jpg")) +
    glob(os.path.join(output_dir_3, "*.jpeg")) +
    glob(os.path.join(output_dir_3, "*.png"))
)
print(f"Jumlah gambar ditemukan: {len(image_paths)}")
if not os.listdir(output_dir_3):
    print("❌ Tidak ada output dari tahap ketiga!")

Jumlah gambar ditemukan: 1


In [32]:
# Langkah 6.1: Konversi output akhir ke JPEG kualitas 100%

from PIL import Image
from glob import glob
import os


os.makedirs(final_output_dir, exist_ok=True)

for img_path in tqdm(image_paths, desc="🧼 Konversi ke JPEG 100%"):
    with Image.open(img_path) as img:
        img = img.convert("RGB")
        filename = os.path.splitext(os.path.basename(img_path))[0] + ".jpg"
        out_path = os.path.join(final_output_dir, filename)
        img.save(out_path, format="JPEG", quality=100, subsampling=0)

print("✅ Semua gambar disimpan ulang sebagai JPEG kualitas 100% di:", final_output_dir)

🧼 Konversi ke JPEG 100%:   0%|          | 0/1 [00:00<?, ?it/s]


IsADirectoryError: [Errno 21] Is a directory: '/content/drive/MyDrive/upscale/output3/20250630_095343.png'

In [ ]:
#Langkah 7: Pengecekan dan verifikasi
import os

#cek ukuran dan format
total_size = sum(os.path.getsize(f) for f in glob(os.path.join(final_output_dir, "*.jpg")))
print(f"📦 Total size semua JPEG 100%: {total_size / (1024 * 1024):.2f} MB")

#validasi format file
for path in glob(os.path.join(final_output_dir, "*.jpg")):
    try:
        with Image.open(path) as img:
            img.verify()
    except Exception as e:
        print(f"❌ Corrupt file: {path} - {e}")